http://pythonhosted.org/svgwrite/

http://web.mit.edu/music21/doc/moduleReference/index.html

In [1]:
from music21 import *

import svgwrite as sw

from IPython.display import display, HTML, Image

import seaborn
from colorsys import hls_to_rgb
from base64 import b16encode

In [50]:
def x_by_cell(cell_num, line_height):
    return (cell_num) * line_height

def y_by_line(line_num, cell_width):
    return (line_num) * cell_width

svg_attrs = {
    'font-family': 'Inconsolata, monospace',
    'font-weight': 'bold',
}

table_attrs = {
    'stroke' : '#000',
    'fill' : '#fff',
    'stroke-width' : '1',
}

cell_attrs = {
    'fill':'none',
    'stroke':'none',
    'stroke-width':'1',
}

pos_attrs = {
    'stroke' : '#000',
    'fill' : '#336',
    'stroke-width' : '1',
}

openstr_attrs = {
    'fill':'none',
    'stroke':'#333',
    'stroke-width':'3',
}

closestr_attrs = {
    'fill':'none',
    'stroke':'#333',
    'stroke-width':'1',
}

circ_attrs = {
    'fill':'#fff',
    'stroke':'white',
    'stroke-width':'0',
}

text_attrs = {
    'font-family': 'Inconsolata, monospace',
    'alignment-baseline' : 'middle',
    'text-anchor' : 'middle',
    'fill' : '#fff',
    'stroke' : 'none',
}

string_num = 4
string_notes = 18

width = 970
cell_width = width/18
cell_height = (cell_width/3)*2
height = cell_height*6

dwg = sw.Drawing(profile='full', size=("{}".format(width), "{}".format(height)), **svg_attrs)

dwg.add(sw.shapes.Rect(insert=(0, y_by_line(1, cell_height)), size=(width, cell_height*4), **table_attrs))

chromatic = scale.ChromaticScale()

#nts = [ pitch.Pitch('e4'),
#        pitch.Pitch('a3'),
#        pitch.Pitch('d3'),
#        pitch.Pitch('g2'),
#       ]

nts = [ chromatic.getPitches('e4', 'a5'),
        chromatic.getPitches('a3', 'd5'),
        chromatic.getPitches('d3', 'g4'),
        chromatic.getPitches('g2', 'c4'),
       ]


pal_hls = seaborn.hls_palette(7, l=.4, s=1).as_hex()

note_colors = dict()
note_names = ['c','d','e','f','g','a','b']
#note_names.sort()

#note_names = chromatic.getPitches('c4','b4')

for idx,note_name in enumerate(note_names):
    note_colors.update({note_name.upper() : pal_hls[idx]})

print(note_colors)

for string in range(string_num):
    string_y = y_by_line(string+1, cell_height)
    #test_pitch = nts[string]
    
    for note in range(string_notes):
        cell_start_x = note*cell_width
        cell_start_y = string_y
        cell_middle_x = cell_start_x + (cell_width/2)
        cell_middle_y = cell_start_y + (cell_height/2)
        circle_radius = ((cell_height/2)/10) * 7
        
        #if note>0:
        #    test_pitch = test_pitch.transpose(1)
        
        grp = sw.container.Group()
        if not nts[string][note].accidental:
            note_color_attrs = circ_attrs.update({'fill' : note_colors[nts[string][note].step],
                                                 'class':'note-{0}'.format(nts[string][note].step)})
        else:
            note_color_attrs = circ_attrs.update({'fill' : '#999'})
        svg_els = [
            sw.shapes.Rect(insert=(cell_start_x, cell_start_y), size=(cell_width, cell_height), **cell_attrs),
            sw.shapes.Circle(center=(cell_middle_x, cell_middle_y), r=circle_radius, **circ_attrs),
            sw.text.Text(nts[string][note].unicodeName, insert=(cell_middle_x, cell_middle_y), **text_attrs),
        ]
        
        for element in svg_els:
            grp.add(element)
            
        dwg.add(grp)
        
    # first element is open
    positions = (
        1, 3, 5, 7, 9, 10, 12
    )
    
    for pos_begin in positions:
        
        # equal for both
        line_x = pos_begin * cell_width
        upper_y = y_by_line(1, cell_height)

        #lower_y = height
        lower_y = y_by_line(1, cell_height) +  (cell_height*4)
        
        half_line = cell_height/2
        
        if pos_begin == 1:
            attrs = openstr_attrs
        else:
            attrs = closestr_attrs
            
        grp = sw.container.Group()
        svg_els = [
            sw.shapes.Line(start=(line_x, upper_y), end=(line_x, lower_y), **attrs),
        ]
        
        for element in svg_els:
            grp.add(element)
            
        dwg.add(grp)

        svg_els = {
            sw.shapes.Rect(insert=(x_by_cell(1,cell_width), half_line), size=(x_by_cell(7,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(3,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(5,cell_width), 0), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(7,cell_width), y_by_line(5, cell_height)+half_line), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(8,cell_width), half_line), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(10,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),
            sw.shapes.Rect(insert=(x_by_cell(12,cell_width), 0), size=(x_by_cell(6,cell_width), half_line), **pos_attrs),

        }
        
        for element in svg_els:
            grp.add(element)

display(HTML(dwg.tostring()))

{'G': '#0068cc', 'C': '#cc0c00', 'D': '#ccbb00', 'B': '#cc00a3', 'A': '#4700cc', 'F': '#00cc81', 'E': '#2ecc00'}
